# Process tracks tables and align time points to Reference Time



In [1]:
import os
import glob
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
from tqdm import tqdm

import networkx as nx
from networkx.drawing.nx_agraph import to_agraph 

from skimage import measure

import telomere
#telomere.py contains different functions being used in this notebook

from functools import partial
import pathos.pools as pp

In [2]:
%matplotlib notebook

In [3]:
# Turn interactive plotting off
plt.ioff()

In [4]:
plt.rcParams.update({'figure.max_open_warning': 0})

# Parameters setting

## Input, output path

In [ ]:
# Define input and output paths
# Note: for Linux and OS path must be end with "/" e.g "/home/Output/ 
# For windows, use "\\" separators e.g. "D:\\_OUTPUT\\
base_input_path = "D:\\_OUTPUT\\_RUNS\\RUN_siNuMa\\CP_Output"
base_output_path = "D:\\_OUTPUT\\_RUNS\\RUN_siNuMa\\Analysis_Output\\"

In [ ]:
# Create "Movies" subfolder
base_output_spath = os.path.join(base_output_path,"Movies")
if not os.path.exists(base_output_spath):
    os.makedirs(base_output_spath)

input_files = glob.glob(os.path.join(base_input_path,"**/*.csv"),recursive=True)
print("nb. of files:",len(input_files))
[print(i,":",input_files[i].split(base_input_path)[1]) for i in range(len(input_files))];

## State transistions

In [ ]:
# Define state transistions
state_labels = ["interphase","prophase","prometaphase","metaphase","anaphase"]
numbers = np.arange(len(state_labels))+1 # the number is assigned automatically in increasing order from 1
state_numbers = pd.Series(index=state_labels,data=numbers)
print(state_numbers);

## Transistion rule graph

In [ ]:
# Initialize graph with multiple directions
G=nx.OrderedMultiDiGraph() # this graph type keeps order of input nodes

# Add nodes
G.add_nodes_from(state_labels);

# Add self transistions
G.add_edges_from(list(zip(state_labels,state_labels)));

# Define transistion here
G.add_edges_from([("interphase",item) for item in ["prophase","prometaphase"]]);
G.add_edges_from([("prophase",item) for item in ["interphase","prometaphase","metaphase"]]);
G.add_edges_from([("prometaphase",item) for item in ["interphase","prophase","metaphase"]]);
G.add_edges_from([("metaphase",item) for item in ["prometaphase","anaphase"]]);
G.add_edges_from([("anaphase",item) for item in ["interphase"]]);

In [ ]:
# Save fig
fig = plt.figure(figsize=(10,7))
ax = fig.add_subplot(111)
pos = nx.circular_layout(G)
nx.draw_networkx(G, pos=pos, ax=ax, width=1, arrowsize=20, 
                 min_source_margin=50, min_target_margin=50,
                 node_shape="s", node_color="none")
fig.savefig(os.path.join(base_output_path,"transistion_rule.png"))
plt.tight_layout();

## Excluded border conditions

In [ ]:
# Define exclude border condition
# criterion can be "bbox" or "circle"
# if criterion is "circle", you can set percentage, e.g. "percentage":0.8 means take 80% of circle area
exclude_borderobjs_conds = {"criterion":"bbox"}

## Alignment conditions

1st order: if the track goes through metapahse then use the last metapahse  as time 0 
2nd order: if the track starts with anaphase, then assign time point 1 
**NOTE: can change this alignment for reversine or prophase as reference time

In [ ]:
# Define rule for aligning time points
align_conds={"state_numbers":[state_numbers["metaphase"],state_numbers["anaphase"]],
             "align_modes":["last","first"],
             "shifts":[0,1]}

## Features

In [ ]:
# Define features will be added after alignment
features = ["ImageNumber","ObjectNumber","TrackObjects_Label",
            "AreaShape_Area",
            "AreaShape_Perimeter",
            "AreaShape_FormFactor",
            "Intensity_IntegratedIntensity_H2B_Smooth",
            "Intensity_IntegratedIntensity_TRF1_Smooth",
            "Intensity_MeanIntensity_H2B_Smooth",
            "Intensity_MeanIntensity_TRF1_Smooth",
            "Mean_Telomere_AreaShape_Area",
            "Mean_Telomere_AreaShape_Perimeter",
            "Mean_Telomere_Distance_Minimum_Nuclei",
            "Mean_Telomere_Distance_Centroid_Nuclei",
            "Mean_Telomere_Intensity_IntegratedIntensity_TRF1_Smooth",
            "Children_Telomere_Count"]

# Test a case specific

This is used to test one file before runing all files. Otherwise, go to next section.

# Parallel computing

In [ ]:
import dill
dill.settings['recurse'] = True

In [ ]:
maxcpu = 18
pool = pp.ProcessPool(min(len(input_files),maxcpu))
print("nb. of allocated cpus:",pool.ncpus)

In [ ]:
def compact_func(f,base_input_path,base_output_spath,
                 features,transistion_graph,
                 nrows_limit,min_nb_timepoints,
                 exclude_borderobjs_conds,align_conds):
    
    # configure output path
    output_path = os.path.join(base_output_spath,f.split(base_input_path)[1].split('.csv')[0])
    
    telomere.process_data(f,output_path,features,transistion_graph,
                          nrows_limit,min_nb_timepoints,
                          exclude_borderobjs_conds,align_conds)

In [ ]:
partial_func = partial(compact_func,
                       base_input_path=base_input_path,base_output_spath=base_output_spath,
                       features=features,transistion_graph=G,
                       nrows_limit=30,min_nb_timepoints=5,
                       exclude_borderobjs_conds=exclude_borderobjs_conds,align_conds=align_conds)

In [ ]:
%%time
result = pool.map(partial_func,input_files)